# < Deep Learning - PART2 TF2 CNNs >

# Ch 5. CNNs Workshop 3 - CIFAR10 : Image Classifier with Data Augmentation & Dropout
2021/10/01


**[ Reference ] :**
+ Keras Documentation : **Train a simple deep CNN on the CIFAR10 small images dataset** https://keras.io/examples/cifar10_cnn/
+ **CIFAR-10 and CIFAR-100 datasets** https://www.cs.toronto.edu/~kriz/cifar.html
+ 李飛飛教授：Convolutional Neural Networks (CNNs / ConvNets)
  (https://cs231n.github.io/convolutional-networks/)
+ `tf.keras.layers.Conv2D`
  (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
  
----

### [ NOTE ] :  Running the following code on Google Colab with GPU !!
-------------------------

## Running TensorFlow 2.0 on Google Colab

In [2]:
from __future__ import print_function

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

2.1.0


## Hyperparameters

In [0]:
batch_size = 32
num_classes = 10
epochs = 50
data_augmentation = True
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

## Loading data...

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Forward Propagation
+ **Building a training model with `tf.keras`**

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

## Backpropagation 

In [0]:
# initiate RMSprop optimizer
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

## Normalization

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Data Augmentation

In [8]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 1563 steps, validate on 10000 samples
Epoch 1/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8522 - accuracy: 0.3156 - val_loss: 1.5686 - val_accuracy: 0.4282
Epoch 2/50
1563/1563 [==============================] - 31s 20ms/step - loss: 1.5820 - accuracy: 0.4212 - val_loss: 1.3962 - val_accuracy: 0.4895
Epoch 3/50
1563/1563 [==============================] - 31s 20ms/step - loss: 1.4621 - accuracy: 0.4668 - val_loss: 1.2855 - val_accuracy: 0.5424
Epoch 4/50
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3848 - accuracy: 0.5029 - val_loss: 1.3087 - val_accuracy: 0.5283
Epoch 5/50
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3199 - accuracy: 0.5261 - val_loss: 1.1952 - val_accuracy: 0.5807
Epoch 6/50
1563/1563 [==============================] - 31s 20ms/step - loss: 1.2646 -

## Saving the model...

In [9]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


## Evaluation

In [10]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 - 1s - loss: 0.7083 - accuracy: 0.7645
Test loss: 0.708253707408905
Test accuracy: 0.7645
